# Logistic regression strategy implementation

In [3]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import datetime
import pandas as pd
import yfinance as yf

In [4]:
def download_data(stock, start, end):
    data = {}
    ticker = yf.download(stock, start, end)
    data['Close'] = ticker['Adj Close']
    return pd.DataFrame(data)

In [5]:
def construct_features(data, lags=2):
    
    # calculate the lagged adjusted closing prices (name = Close)
    for i in range(0, lags):
        data['Lag%s' % str(i+1)] = data['Close'].shift(i+1)
    
    print(data)

In [ ]:
if __name__ ='__main__':
    start_date = datetime.datetime(2017, 1, 1)
    end_date   = datetime.datetime(2018, 1, 1)
    
    stock_data = download_data('IBM', start_date, end_date)
    construct_fea